## Code de florette (EN COURS)

En premier il y a le code pour la comparaison.

Aprés il y a plein d'autres trucs qui pourraient être utiles:
- des test de tokenisation
- des test de formation de n grams
- des tests d'utilisation de FastText (il en parle dans le paper dont le nom est marqué u peu plus bas) (c'est un fail)

Encore plus bas:
- des test de comparaisons sur plein de trucs (random, trus qui se ressemblent, grandes/petites phrases, synonymes) pour voir comment ça marche

#### Mount the Google Drive in order to access to the files which are located on our drive


In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


#### Install and import every packages that will be necessary for the analysis

In [2]:
!pip install pickle-mixin
!pip install pyarrow
!pip install pathlib


import pyarrow
import pandas as pd
import numpy as np
import pathlib
from datetime import datetime
import sys
import bz2
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

### Imports, et import du model qu'on utilise ( pour l'instant all-MiniLM-L6-v2 )

In [1]:
#si besoin
#pip install -U sentence-transformers
#pip install gensim

In [48]:
#NLP libraries
import nltk
from nltk import word_tokenize
from gensim.models import FastText

In [229]:
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer

#import du model qu'on utilise, je prend all-MiniLM-L6-v2. là j'ai le model en local, mais on eut le prendre direct du net, c'est automoatisé (juste là au moment où je core il y à un bug généralisé, genre plus personne n'y a accès en ligne)
#model = SentenceTransformer(r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\pretrained_model') #with model all-MiniLM-L6-v2' # or use 'bert-base-nli-mean-tokens' (deprecated), or 'all-mpnet-base-v2', or 'multi-qa-mpnet-base-dot-v1', 'multi-qa-distilbert-cos-v1'
model = SentenceTransformer('all-MiniLM-L6-v2')


# j'ai testé plysieurs models
# all-MiniLM-L6-v2: ça passe, treshold à 0.6 ?
#all-mpnet-base-v2: ne se télécharge pas
#multi-qa-mpnet-base-dot-v1: ne va pas, tous les scores sont trés haut meme pour truc contraires
#multi-qa-distilbert-cos-v1: mouai


### Comparaison des citations (morceau par morceau) aux expressions du paper, et création d'un dataframe avec qids, quote, sim_score_expression1, sim_score_expression2, ..., sim_score_expressionN.

Je decoupe les citations. Pour chaque expression: je compare chaque morceau de la citation avec l'expression, ça donne un score de similarité pour chaque morceau: on prend le max (ou bien faire un teshold ? à voir).  







#### Basé sur la technique du paper Sentence Similarity Techniques for Short vs Variable Length Text using Word Embeddings


#### Extract a small part of the whole quotes file to test (j'ai pas le vrai fichier pre procéssé, c'est pour ça qu'il y a des [ ] dans les citations, mais ça marchera pareil avec le bon fichier

In [266]:
#Extract a small part of the whole file to test 
path = r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\quotes-processed.json.bz2' #path de mon ordi
quotes = pd.DataFrame(columns=('qids','quotation'))
i = 0
with bz2.open(path, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) 
    #print(instance)
    #print(instance['quotation'])
    i = i + 1
    quotes = quotes.append({'qids': instance['qids'], 'quotation': instance['quotation']}, ignore_index=True)
    #print(i)
    if i == 10:
      break

#### append some quotation for testing the efficacy

In [267]:
quotes = quotes.append({'qids': 'Qtest1', 'quotation': 'I know it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest2', 'quotation': 'I dont know if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest3', 'quotation': 'I am sure it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest4', 'quotation': 'I am not sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest5', 'quotation': 'I dont eat an apple'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest6', 'quotation': 'I pretty sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest7', 'quotation': 'I was sure if it is a fool'}, ignore_index=True)





#### create a dataframe de test with the expression we want to compare, and their score (remplacer aprés avec le vrai)

In [268]:
#create a dataframe with the ewpression we want to compare, and their score
expressions = pd.DataFrame({'expression': ["I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"], 'expression_score': [2.91, 3.02, 3.34, 3.48, 6.02, 6.45]})
expressions.head(3)

,expression,expression_score
0,I'm not sure,2.91
1,I dont know,3.02
2,I suppose,3.34


### Comparaison

#### Chunker les citations pour découper en un certain nombre de mots (j'ai pas trouvé de méthode optimale, alors je tokenize en mots, puis je join le nombre de mots qu'on veut

In [269]:
#First, create dataframe of the expressions tokenized and their confidence score
quotes_tokenized = pd.DataFrame(columns = ['qids','quote_token'])
for index, row in quotes.iterrows():
    quotes_tokenized = quotes_tokenized.append({'qids': row.qids, 'quote_token': nltk.word_tokenize(row.quotation)}, ignore_index=True)
quotes_tokenized.head(3)


,qids,quote_token
0,[Q367796],"[Department, of, Homeland, Security, was, livi..."
1,[Q20684375],"[I, met, them, when, they, just, turned, 4, an..."
2,[Q5268447],"[The, delay, will, have, an, impact, on, Sloug..."


In [270]:
#Then reform chunk of n words

n = 4 #number of word in a chunk

quotes_chunks = pd.DataFrame(columns = ['qids','quote_chunks']) #initialize dataframe 


for index, row in quotes_tokenized.iterrows():
    lenght = len(row.quote_token) 
    list_chunk = []
    for i in range(lenght - n + 1):
        chunk = " ".join(row.quote_token[i : i+n-1])
        list_chunk.append(chunk)
    #print(list_chunk)
    #print('\n')
    quotes_chunks = quotes_chunks.append({'qids': row.qids, 'quote_chunks': list_chunk}, ignore_index=True)

        
quotes_chunks.head()
    
    

,qids,quote_chunks
0,[Q367796],"[Department of Homeland, of Homeland Security,..."
1,[Q20684375],"[I met them, met them when, them when they, wh..."
2,[Q5268447],"[The delay will, delay will have, will have an..."
3,[Q4864119],"[The scheme treats, scheme treats addiction, t..."
4,[Q816459],"[These actions will, actions will allow, will ..."


#### Compare the citations chunk with the expressions and add the comparison score = cosinus similarity (max score for the different chunk) in a new column

In [273]:
quotes_similarity = pd.DataFrame(columns=('qids','quotation', "I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"))
df_max_cos_sim = pd.DataFrame(index=("I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"))

for index, row in quotes_chunks.iterrows(): #itere sur les citations
    
    for index_exp, row_exp in expressions.iterrows(): #pour chaque expression
        list_cis_sim = [] #list des cosine similarity score pour tous les chunk (on gardera que le max)
        exp_to_compare = model.encode(row_exp.expression)
        
        for chunk in row.quote_chunks: #itere sur les chunk de citation
            chunk_to_compare = model.encode(chunk)
            cos_sim = cosine_similarity([chunk_to_compare], [exp_to_compare]) #absolument pas sure que je l'utilise comme il faut
            #print(cos_sim)
            list_cis_sim.append(cos_sim)
        
        max_cos_sim = max(list_cis_sim)
        df_max_cos_sim.loc[row_exp.expression, 0] = max_cos_sim
    
    quotes_similarity = quotes_similarity.append({'qids': row.qids, 'quotation': row.quote_chunks, "I'm not sure": df_max_cos_sim.loc["I'm not sure", 0], "I dont know": df_max_cos_sim.loc["I dont know", 0], "I suppose": df_max_cos_sim.loc["I suppose", 0], "I think": df_max_cos_sim.loc["I think", 0], "I'm sure": df_max_cos_sim.loc["I'm sure", 0], "I know": df_max_cos_sim.loc["I know", 0]}, ignore_index=True)

#quotes_similarity.head(12)
        


In [275]:
display(quotes_similarity.iloc[10:20])

,qids,quotation,I'm not sure,I dont know,I suppose,I think,I'm sure,I know
10,Qtest1,"[I know it, know it is, it is a]",0.287719,0.251294,0.306325,0.366793,0.391116,0.818253
11,Qtest2,"[I dont know, dont know if, know if it, if it ...",0.759200,1.000000,0.420749,0.447580,0.437839,0.355956
12,Qtest3,"[I am sure, am sure it, sure it is, it is a]",0.525282,0.418564,0.560582,0.469052,0.946308,0.458801
13,Qtest4,"[I am not, am not sure, not sure if, sure if i...",0.850227,0.683103,0.390731,0.468090,0.503178,0.344952
14,Qtest5,"[I dont eat, dont eat an]",0.161471,0.195348,0.100146,0.065757,0.159083,0.224308
15,Qtest6,"[I pretty sure, pretty sure if, sure if it, if...",0.630606,0.520012,0.644584,0.654895,0.793351,0.344952
16,Qtest7,"[I was sure, was sure if, sure if it, if it is...",0.428967,0.295433,0.419736,0.366793,0.767583,0.485900


In [242]:
#avec le premier model testé, NE PAS RUNNER !!!!!!!!!!!!!
quotes_similarity.head(20)

,qids,quotation,I'm not sure,I'dont know,I'suppose,I'think,I'sure,I'know
0,[Q367796],"[Department of Homeland, of Homeland Security,...",0.303232,0.297696,0.217772,0.228274,0.242716,0.199665
1,[Q20684375],"[I met them, met them when, them when they, wh...",0.230303,0.233470,0.266534,0.275259,0.261234,0.249393
2,[Q5268447],"[The delay will, delay will have, will have an...",0.412485,0.323660,0.517528,0.621261,0.326985,0.336410
3,[Q4864119],"[The scheme treats, scheme treats addiction, t...",0.272484,0.241025,0.238938,0.254450,0.240250,0.294437
4,[Q816459],"[These actions will, actions will allow, will ...",0.283460,0.298020,0.223775,0.182380,0.205218,0.217203
5,[Q30164281],"[1 . FM, . FM is, FM is entitled, is entitled ...",0.292810,0.238106,0.322083,0.251693,0.361709,0.255408
6,[Q56255401],"[11 straight weeks, straight weeks of, weeks o...",0.185044,0.168329,0.139400,0.201659,0.170129,0.179680
7,[Q26923564],"[2019 was a, was a landmark, a landmark year, ...",0.239804,0.236918,0.169898,0.239955,0.213944,0.205041
8,[Q4749380],"[7pm is when, is when most, when most hospital...",0.269845,0.314072,0.339756,0.267074,0.492714,0.604759
9,[Q970000],"[A city is, city is not, is not an, not an acc...",0.310569,0.282475,0.228700,0.236583,0.201255,0.277545


In [185]:
for index, row in quotes_similarity.iterrows():
    #display(row)

SyntaxError: unexpected EOF while parsing (<ipython-input-185-7dffd709778c>, line 2)

In [159]:
df_max_cos_sim

,0
I'm not sure,0.269845
I'dont know,0.314072
I'suppose,0.339756
I'think,0.267074
I'sure,0.492714
I'know,0.604759


In [171]:
quotes.quotation.iloc[2]

'The delay will have an impact on Slough but that might be mitigated by the fact we are going to have this Western Rail Link to Heathrow. It looks like that may come in sooner than Crossrail.'

In [ ]:
quotes_chunks

### Autres trucs que j'ai pas utilisé mais qui peuvent être utile, genre tokenisation

#### Tokenisation of the expressions

In [46]:
#create dataframe of the expressions tokenized and their confidence score
expressions_tokenized = pd.DataFrame(columns = ['expression_token', 'score'])
for index, row in expressions.iterrows():
    expressions_tokenized = expressions_tokenized.append({'expression_token': nltk.word_tokenize(row.expression), 'score': row.expression_score}, ignore_index=True)
expressions_tokenized.head(3)


,expression_token,score
0,"[I, 'm, not, sure]",2.91
1,"[I'dont, know]",3.02
2,[I'suppose],3.34


#### Tokenisation of the quotations, en formant des n grams

In [97]:
from nltk import ngrams

n = 4 # n_gram
quotes_n_grams = pd.DataFrame(columns = ['quoteID', 'quote_n_grams'])

for index, row in quotes.iterrows():
    n_grams = ngrams(row.quotation.split(), n)
    quotes_n_grams = quotes_n_grams.append({'quoteID': row.quoteID, 'n_grams': n_grams},  ignore_index=True)
    
i = 0
for index, row in quotes_n_grams.iterrows():
    print(i)
    i=i+1
    for grams in row.n_grams:
        print(grams)
    if i > 1 : break


0
('[', 'Department', 'of', 'Homeland')
('Department', 'of', 'Homeland', 'Security')
('of', 'Homeland', 'Security', ']')
('Homeland', 'Security', ']', 'was')
('Security', ']', 'was', 'livid')
(']', 'was', 'livid', 'and')
('was', 'livid', 'and', 'strongly')
('livid', 'and', 'strongly', 'urged')
('and', 'strongly', 'urged', 'to')
('strongly', 'urged', 'to', 'have')
('urged', 'to', 'have', 'the')
('to', 'have', 'the', 'agenda')
('have', 'the', 'agenda', 'pulled.')
1
('[', 'I', 'met', 'them')
('I', 'met', 'them', ']')
('met', 'them', ']', 'when')
('them', ']', 'when', 'they')
(']', 'when', 'they', 'just')
('when', 'they', 'just', 'turned')
('they', 'just', 'turned', '4')
('just', 'turned', '4', 'and')
('turned', '4', 'and', '7.')
('4', 'and', '7.', 'They')
('and', '7.', 'They', 'were')
('7.', 'They', 'were', 'little.')
('They', 'were', 'little.', 'They')
('were', 'little.', 'They', 'felt')
('little.', 'They', 'felt', 'like')
('They', 'felt', 'like', 'my')
('felt', 'like', 'my', 'full-blown

### Trucs randoms, tests, code qui pourrait être utiles

#### exemples de truc pour faire des n grams

In [71]:
#exemple, not optimal, generate bigram and trigram

from nltk import ngrams
sentence = ['i have an apple', 'i like apples so much']

for i in range(len(sentence)):
    for n in range(2, 4):
        n_grams = ngrams(sentence[i].split(), n)
        for grams in n_grams:
            print(grams)

('i', 'have')
('have', 'an')
('an', 'apple')
('i', 'have', 'an')
('have', 'an', 'apple')
('i', 'like')
('like', 'apples')
('apples', 'so')
('so', 'much')
('i', 'like', 'apples')
('like', 'apples', 'so')
('apples', 'so', 'much')


In [75]:
#autre exemple de generatipn de bigram

input_list = ['all', 'this', 'happened', 'more', 'or', 'less']

def find_bigrams(input_list):
    bigram_list = []
    for i in range(len(input_list)-1):
        bigram_list.append((input_list[i], input_list[i+1]))
    return bigram_list

find_bigrams(input_list)

[('all', 'this'),
 ('this', 'happened'),
 ('happened', 'more'),
 ('more', 'or'),
 ('or', 'less')]

#### test d'utilisation de FastTest

In [70]:
#pip install fasttext-win

In [69]:
>>> import fasttext.util
>>> fasttext.util.download_model('en', if_exists='ignore')  # English
>>> ft = fasttext.load_model('cc.en.300.bin')

ModuleNotFoundError: No module named 'fasttext'

In [64]:
# Load pretrained fastText word embeddings python with gensim
from gensim.models.fasttext import load_facebook_model
pretrained_fastText_en = load_facebook_model('pretrined fastText model/cc.en.300.bin.gz')

FileNotFoundError: [Errno 2] No such file or directory: 'cc.en.300.bin.gz'

In [54]:
#en fait ça c'est pour trained un model, mais c'est pas ce qu'on veut faire ducoup
exp_model = FastText(expressions_tokenized.expression_token, vector_size=100, window=4, min_count=1, workers=4,sg=1)

In [61]:
#exp_model.wv['computer']

### Tests de comparaison : avec des trucs randoms, avec des citations

#### test de comparaison avec des trucs randoms

In [4]:
#test de comparaison semantique avec des trucs randoms

sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]

from sentence_transformers import SentenceTransformer

model = SentenceTransformer(r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\pretrained_model') #'bert-base-nli-mean-tokens'

sentence_embeddings = model.encode(sentences)

from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[1:]
)

array([[0.22059387, 0.47637683, 0.22743046]], dtype=float32)

#### test de comparaison de quelques unes de nos citations entières avec les expressions. 

Ce n'est pas une bonnemethode, on va devoir découper les phrases en morceau.

In [8]:
#test de comparaison semantique de quelques une de nos citations avec une des phrases types du paper
#j'ai fait comme dans un exemple de BERT que j'ai trouvé sur internet. j'ai pris le pretrained model all-MiniLM-L6-v2


#note: normalement pas besoin d'avoir le model sur l'ordi, il suffit de mettre le nom 'all-MiniLM-L6-v2' et SentenceTransformer va le chercher tout seul sur internet, mais là il y a un bug genre ajd, 
#donc j'ai téléchargé le model sur mon ordi et mis le path
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\pretrained_model') #with model all-MiniLM-L6-v2' # or use 'bert-base-nli-mean-tokens'


#Extract a small part of the whole file to test 
path = r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\quotes-2020-precessed.json.bz2' #path de mon ordi
test = pd.DataFrame(columns=('qids','quotation'))
i = 0
with bz2.open(path, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) 
    print(instance)
    #print(instance['quotation'])
    i = i + 1
    test = test.append({'quoteID': instance['quoteID'], 'quotation': instance['quotation']}, ignore_index=True)
    #print(i)
    if i == 10:
      break

#embedding des 10 citations
sentence_embeddings = model.encode(test['quotation'])

#comparaison avec une des phrase du paper: cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
to_compare = ["I'm sure it's"]
to_compare_embedding = model.encode(to_compare)

cosine_similarity(
    [to_compare_embedding[0]],
    sentence_embeddings[0:]
)

#l'output ets un array des coefficients de similarité (là ils osnt tous ba vu qu'aucune citation n'a un truc prche de I'm sure it's)

{'quoteID': '2020-01-16-000088', 'quotation': '[ Department of Homeland Security ] was livid and strongly urged to have the agenda pulled.', 'speaker': 'Sue Myrick'}
{'quoteID': '2020-01-24-000168', 'quotation': '[ I met them ] when they just turned 4 and 7. They were little. They felt like my full-blown stepkids.', 'speaker': 'Meghan King Edmonds'}
{'quoteID': '2020-01-17-000357', 'quotation': '[ The delay ] will have an impact [ on Slough ] but that might be mitigated by the fact we are going to have this Western Rail Link to Heathrow. It looks like that may come in sooner than Crossrail.', 'speaker': 'Dexter Smith'}
{'quoteID': '2020-04-02-000239', 'quotation': '[ The scheme ] treats addiction as an illness and the results so far have been extremely encouraging.', 'speaker': 'Barry Coppinger'}
{'quoteID': '2020-03-19-000276', 'quotation': '[ These ] actions will allow households who have an FHA-insured mortgage to meet the challenges of COVID-19 without fear of losing their homes, a

array([[ 0.01433591,  0.02702463,  0.04557954,  0.07573312, -0.02040301,
         0.09890234,  0.07345733,  0.0720223 ,  0.02289211,  0.05893324]],
      dtype=float32)

#### encore des tests pour voir si ça marche bien

In [22]:
#test de comparaison avec le début de la premiere citation pour checker qu'elle a alors bien un score élevé
to_compare_2 = ["Department of Homeland Security was livid and"]
to_compare_2_embedding = model.encode(to_compare_2)

cosine_similarity(
    [to_compare_2_embedding[0]],
    sentence_embeddings[0:]
)
#le premier est bien plus élevé

array([[0.7573065 , 0.02370122, 0.0167593 , 0.07905838, 0.12434015,
        0.13122094, 0.01477368, 0.07131456, 0.1100079 , 0.1029016 ]],
      dtype=float32)

In [29]:
#test de comparaison avec 2 trucs trés proches
to_compare_3 = ["I'm sure"]
to_compare_3_embedding = model.encode(to_compare_3)
sentence_2 = ["I'm sure it will be OK"]
sentence_2_embedding = model.encode(sentence_2)

cosine_similarity(
    [to_compare_3_embedding[0]],
    sentence_2_embedding[0:]
)

array([[0.17984062]], dtype=float32)

In [8]:
#test de comparaison avec 2 synonymes
to_compare_3 = ["I can be wrong"]
to_compare_3_embedding = model.encode(to_compare_3)
sentence_2 = ["I am mistaken coucou love"] #vrae faf feafq fezfaZF vzev ffreyhju j'ythgz azerazefg rt rt  r zeardcre grtuhy thzrg area etzttrh trhtrbg er ezt ery gt areer fraef dgc r e zger 
sentence_2_embedding = model.encode(sentence_2)

cosine_similarity(
    [to_compare_3_embedding[0]],
    sentence_2_embedding[0:]
)

array([[0.19524246]], dtype=float32)